<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/annotation_import/video.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>

<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/annotation_import/video.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td> 

# Video Annotation Import

* Annotations must be created and uploaded using NDJSON
* Supported annotations that can be uploaded through the SDK:
    * Bounding box
    * Point
    * Polyline 
    * Radio classifications 
    * Checklist classifications 
* **NOT** supported:
    * Polygons 
    * Segmentation masks
    * Free form text classifications

Please note that this list of unsupported annotations only refers to limitations for importing annotations. For example, when using the Labelbox editor, segmentation masks can be created and edited on video assets.

### Setup

In [ ]:
# !pip install -q 'labelbox[data]'

In [ ]:
import labelbox as lb
import labelbox.types as lb_types
import uuid

### Replace with your API key 
Guides on [Create an API key](https://docs.labelbox.com/docs/create-an-api-key)

In [ ]:
# Add your api key
API_KEY=""
client = lb.Client(api_key=API_KEY)

## Supported annotations for video


In [ ]:
######## Bounding box  ###########
# Python Annotation 
bbox_annotation = [
  lb_types.VideoObjectAnnotation(
    name = "bbox_video",  
    keyframe=True,
    frame=13,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=146.0, y=98.0), # Top left
          end=lb_types.Point(x=382.0, y=341.0), # Bottom right
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "bbox_video",  
    keyframe=True,
    frame=15,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=146.0, y=98.0), # Top left
          end=lb_types.Point(x=382.0, y=341.0), # Bottom right
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "bbox_video",  
    keyframe=True,
    frame=19,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=146.0, y=98.0), # Top left
          end=lb_types.Point(x=382.0, y=341.0), # Bottom right
      )
  )
]


# NDJSON
bbox_annotation_ndjson = {
    "name" : "bbox_video",
    "segments" : [{
        "keyframes" : [
            {
              "frame": 13,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }  
           },
           {
              "frame": 15,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }  
           },
           {
              "frame": 19,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }  
           }
        ]
      }
    ]
}

In [ ]:
######## Point ########

point_annotation = [
    lb_types.VideoObjectAnnotation(
        name = "point_video",
        keyframe=True,
        frame=17,
        value = lb_types.Point(x=660.134, y=407.926),
        )
]

#NDJSON
point_annotation_ndjson = {
    "name": "point_video", 
    "segments": [{
        "keyframes": [{
            "frame": 17,
            "point" : {
                "x": 660.134 ,
                "y": 407.926
            }
        }]
    }] 
}

In [ ]:
######## Polyline ########
polyline_annotation = [
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=5,
    segment_index=0,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=12,
    segment_index=0,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=20,
    segment_index=0,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=24,
    segment_index=1,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=45,
    segment_index=1,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  )
  
]


polyline_frame_annotation_ndjson = {
  "name": "line_video_frame", 
  "segments": [
      {
        "keyframes": [
          {
            "frame": 5,
            "line": [{
              "x": 680,
              "y": 100
            },{
              "x": 100,
              "y": 190
            },{
              "x": 190,
              "y": 220
            }]
          },
          {
            "frame": 12,
            "line": [{
              "x": 680,
              "y": 280
            },{
              "x": 300,
              "y": 380
            },{
              "x": 400,
              "y": 460
            }]
          },
          {
            "frame": 20,
            "line": [{
              "x": 680,
              "y": 180
            },{
              "x": 100,
              "y": 200
            },{
              "x": 200,
              "y": 260
            }]
          }
        ]
      },
      {
        "keyframes": [
          {
            "frame": 24,
            "line": [{
              "x": 300,
              "y": 310
            },{
              "x": 330,
              "y": 430
            }]
          },
          {
            "frame": 45,
            "line": [{
              "x": 600,
              "y": 810
            },{
              "x": 900,
              "y": 930
            }]
          }
        ]
      }
    ]
}

In [ ]:
######## Frame base classifications ########


radio_annotation = [
    lb_types.VideoClassificationAnnotation(
        name="radio_class", 
        frame=9,
        segment_index=0,
        value=lb_types.Radio(answer = lb_types.ClassificationAnswer(name = "first_radio_answer"))
    ),
    lb_types.VideoClassificationAnnotation(
        name="radio_class", 
        frame=15,
        segment_index=0,
        value=lb_types.Radio(answer = lb_types.ClassificationAnswer(name = "first_radio_answer"))
    )
]

checklist_annotation= [
    lb_types.VideoClassificationAnnotation(
        name="checklist_class",
        frame=29,
        segment_index=0,
        value=lb_types.Checklist(
            answer = [
                lb_types.ClassificationAnswer(
                    name = "first_checklist_answer"
                )
            ]
            )
        ),
    lb_types.VideoClassificationAnnotation(
        name="checklist_class", 
        frame=35,
        segment_index=0,
        value=lb_types.Checklist(
            answer = [
                lb_types.ClassificationAnswer(
                    name = "first_checklist_answer"
                )
            ]
            )
        ),
    lb_types.VideoClassificationAnnotation(
        name="checklist_class", 
        frame=39, 
        segment_index=1,
        value=lb_types.Checklist(
            answer = [
                lb_types.ClassificationAnswer(
                    name = "second_checklist_answer"
                )
            ]
            )
        ),
    lb_types.VideoClassificationAnnotation(
        name="checklist_class", 
        frame=45, 
        segment_index=1,
        value=lb_types.Checklist(
            answer = [
                
                lb_types.ClassificationAnswer(
                    name = "second_checklist_answer"
                )
            ]
            )
        )
]


## NDJSON

frame_radio_classification_ndjson = {
    "name": "radio_class", 
    "answer": { "name": "first_radio_answer", "frames": [{"start": 9, "end": 15}]}
}

## frame specific
frame_checklist_classification_ndjson = {
    "name": "checklist_class", 
    "answer": [
        { "name": "first_checklist_answer" , "frames": [{"start": 29, "end": 35 }]},
        { "name": "second_checklist_answer", "frames": [{"start": 39, "end": 45 }]} 
  ]      
}





In [ ]:
##### Global Classifications ####### 

## For global classifications use ClassificationAnnotation
global_radio_annotation = [lb_types.ClassificationAnnotation(
    name="radio_class_global",
    value=lb_types.Radio(answer = lb_types.ClassificationAnswer(name = "first_radio_answer"))
)]

global_checklist_annotation=[lb_types.ClassificationAnnotation(
  name="checklist_class_global", 
  value=lb_types.Checklist(
      answer = [
        lb_types.ClassificationAnswer(
            name = "first_checklist_answer"
        ), 
        lb_types.ClassificationAnswer(
            name = "second_checklist_answer"
        )
      ]
    )
 )]

global_radio_classification_ndjson = {
    "name": "radio_class_global", 
    "answer": { "name": "first_radio_answer"}
}


global_checklist_classification_ndjson = {
    "name": "checklist_class_global", 
    "answer": [
        { "name": "first_checklist_answer" },
        { "name": "second_checklist_answer"} 
  ]      
}

In [ ]:
########## Nested Global Classification ########### 

nested_radio_classification = {
  'name': 'radio_question_nested',
  'answer': {'name': 'first_radio_question'},
  'classifications' : [
    {'name': 'sub_question_radio', 'answer': {'name': 'sub_answer'}}
   ]
}

nested_checklist_annotation_ndjson = {
  "name": "nested_checklist_question",
  "answer": [{
      "name": "first_checklist_answer",
      "classifications" : [
        {
          "name": "sub_checklist_question",
          "answer": {"name": "first_sub_checklist_answer"}
        }
      ]
  }]
}

In [ ]:
########## Classifications under frame base tools ##########

# Frame base nested classifications do not support using the feature's name to extract ontology features. 
# For this single case we are going to use the classification's featureSchemaId and the answers' featureSchemaId 
# We will update the annotation object with the featureSchemaIds on step 5 after we create the ontology in step 2


frame_bbox_with_checklist_subclass_ndjson = {
    "name": "bbox_class",
    "segments": [{
        "keyframes": [
            {
            "frame": 10,
            "bbox": {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              },
            "classifications" : [
              {'schemaId' : '', 'answer' : {'schemaId': '' }}
            ]     
          },
          {  
          "frame": 11,
            "bbox": {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              },
            "classifications" : [
              {'schemaId' : '', 'answer' : {'schemaId': '' }}
            ]  
          },
          {  
          "frame": 13,
            "bbox": {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              },
            "classifications" : [
              {'schemaId' : '', 'answer' : {'schemaId': '' }}
            ]  
          }
        ]
      }
    ]
}

In [ ]:
##### Raster Segmentation ########
import cv2


instance_uri_1 = 'https://storage.labelbox.com/cjhfn5y6s0pk507024nz1ocys%2F1d60856c-59b7-3060-2754-83f7e93e0d01-1?Expires=1666901963361&KeyName=labelbox-assets-key-3&Signature=t-2s2DB4YjFuWEFak0wxYqfBfZA'
instance_uri_2 = 'https://api.labelbox.com/masks/feature/clfr7k2sx00033b6ja7dxd7ap/1'
instance_uri_3 = "https://storage.labelbox.com/cjhfn5y6s0pk507024nz1ocys%2F1d60856c-59b7-3060-2754-83f7e93e0d01-1?Expires=1666901963361&KeyName=labelbox-assets-key-3&Signature=t-2s2DB4YjFuWEFak0wxYqfBfZA"
# instance_uri_5 = 'https://storage.labelbox.com/cjhfn5y6s0pk507024nz1ocys1%2F1d60856c-59b7-3060-2754-83f7e93e0d01-1?Expires=1666901963361&KeyName=labelbox-assets-key-3&Signature=t-2s2DB4YjFuWEFak0wxYqfBfZA'

video_mask_annotation=[
    lb_types.VideoMaskAnnotation(
        frames=[
            lb_types.MaskFrame(index=10, instance_uri=instance_uri_2)
        ],
        instances=[
            lb_types.MaskInstance(color_rgb=(0,0,0), name="video_mask")
        ]     
    )
]



video_mask_annotation_ndjson = {
    # 'name': 'video_mask',
    'masks': {
        'frames': [{
            'index': 10,
            'instanceURI': instance_uri_3
        }],
        'instances': [
            {
                'colorRGB': (239, 147, 0),
                # 'schemaId': 'clfr5sp7a038b07yg99c61tdb'
                'name': 'video_mask',
            }
        ]
    }
}

# video_mask_annotation_ndjson = {
#     'name': 'video_mask',
#     'masks': {
#         'frames': [{
#             'index': 1,
#             'instanceURI': instance_uri_2
#         }],
#         'instances': [
#             {
#                 'colorRGB': (0, 0, 0),
#                 'name': 'mask1'
#             }
#         ]
#     }
# }

## Upload Annotations - putting it all together

### Step 1: Import data rows into Catalog

In [ ]:
global_key = "sample-video-2.mp4"
asset = {
    "row_data": "https://storage.googleapis.com/labelbox-datasets/video-sample-data/sample-video-2.mp4", 
    "global_key": global_key,
    "media_type": "VIDEO"
}

dataset = client.create_dataset(name="video_demo_dataset")
task = dataset.create_data_rows([asset])
task.wait_till_done()
print("Errors :",task.errors)
print("Failed data rows:" ,task.failed_data_rows)

There are errors present. Please look at `task.errors` for more details


Errors : Duplicate global keys found: sample-video-2.mp4
Failed data rows: [{'message': 'Duplicate global keys found: sample-video-2.mp4', 'failedDataRows': [{'globalKey': 'sample-video-2.mp4', 'rowData': 'https://storage.googleapis.com/labelbox-datasets/video-sample-data/sample-video-2.mp4', 'attachmentInputs': []}]}]


### Step 2: Create/select an ontology
Your project should have the correct ontology setup with all the tools and classifications supported for your annotations, and the tool names and classification instructions should match the `name` fields in your annotations to ensure the correct feature schemas are matched.

For example, when we create the bounding box annotation above, we provided the `name` as `bbox_video`. Now, when we setup our ontology, we must ensure that the name of my bounding box tool is also `bbox_video`. The same alignment must hold true for the other tools and classifications we create in our ontology.


[Documentation for reference ](https://docs.labelbox.com/reference/import-text-annotations)

In [ ]:
ontology_builder = lb.OntologyBuilder(
    tools=[
        lb.Tool(tool=lb.Tool.Type.BBOX, name="bbox_video"),
        lb.Tool(tool=lb.Tool.Type.POINT, name="point_video"),
        lb.Tool(tool=lb.Tool.Type.LINE, name="line_video_frame"),
        lb.Tool(tool=lb.Tool.Type.RASTER_SEGMENTATION, name="video_mask"),
        lb.Tool(
          tool=lb.Tool.Type.BBOX, name="bbox_class",
          classifications=[
            lb.Classification(
              class_type=lb.Classification.Type.RADIO,
              name="bbox_radio",
              scope = lb.Classification.Scope.INDEX,
              options=[
                lb.Option(value="bbox_radio_answer_1"),
                lb.Option(value="bbox_radio_answer_2"),
                lb.Option(value="bbox_radio_answer_3")
              ]
            )
          ]
        )
    ],
    classifications=[ 
        lb.Classification(
            class_type=lb.Classification.Type.CHECKLIST,
            name="checklist_class",
            scope = lb.Classification.Scope.INDEX, ## Need to defined scope for frame classifications
            options=[ 
                lb.Option(value="first_checklist_answer"),
                lb.Option(value="second_checklist_answer")
            ]
        ),
        lb.Classification(
            class_type=lb.Classification.Type.RADIO,
            name="radio_class",
            scope = lb.Classification.Scope.INDEX,
            options=[ 
                lb.Option(value="first_radio_answer"),
                lb.Option(value="second_radio_answer")
            ]
        ),
         lb.Classification(
              class_type=lb.Classification.Type.RADIO,
              name="radio_question_nested",
              options=[
                  lb.Option("first_radio_question",
                        options=[
                            lb.Classification(
                                class_type=lb.Classification.Type.RADIO,
                                name="sub_question_radio",
                                options=[lb.Option("sub_answer")]
                            )
                        ]
                  )
              ] 
        ),
        lb.Classification(
          class_type=lb.Classification.Type.CHECKLIST,
          name="nested_checklist_question",
          options=[
              lb.Option("first_checklist_answer",
                options=[
                  lb.Classification(
                      class_type=lb.Classification.Type.CHECKLIST,
                      name="sub_checklist_question",
                      options=[lb.Option("first_sub_checklist_answer")]
                  )
              ]
            )
          ]
        ),
        lb.Classification(
          class_type=lb.Classification.Type.RADIO, 
          name="radio_class_global",
          options=[ 
                lb.Option(value="first_radio_answer"),
                lb.Option(value="second_radio_answer")
            ]
        ),
        lb.Classification(
          class_type=lb.Classification.Type.CHECKLIST,
          name="checklist_class_global",
          options=[
                lb.Option(value="first_checklist_answer"),
                lb.Option(value="second_checklist_answer")
          ]
        )
    ]  
)

ontology = client.create_ontology("Ontology Video Annotations", ontology_builder.asdict(), media_type=lb.MediaType.Video)

### Step 3: Create a labeling project 
Connect the ontology to the labeling project.

In [ ]:
# Project defaults to batch mode with benchmark quality settings if this argument is not provided
# Queue mode will be deprecated once dataset mode is deprecated

project = client.create_project(name="video_project_demo",
                                    queue_mode=lb.QueueMode.Batch,
                                    media_type=lb.MediaType.Video)

## connect ontology to your project
project.setup_editor(ontology)

### Step 4: Send a batch of data rows to the project

In [ ]:
# Create batches

# Create a batch to send to your MAL project
batch = project.create_batch(
  "first-batch-video-demo2", # Each batch in a project must have a unique name
  global_keys=[global_key], # A paginated collection of data row objects
  priority=5 # priority between 1(Highest) - 5(lowest)
)

print("Batch: ", batch)

Batch:  <Batch {
    "consensus_settings_json": "{\"numberOfLabels\":1,\"coveragePercentage\":0}",
    "created_at": "2023-03-27 19:22:44+00:00",
    "name": "first-batch-video-demo2",
    "size": 0,
    "uid": "c03fc090-ccd4-11ed-accc-a33d5c85b35a",
    "updated_at": "2023-03-27 19:22:44+00:00"
}>


### Step 5: Create the annotations payload 
Create the annotations payload using the snippets of code above.

Labelbox supports two formats for the annotations payload: NDJSON and Python Annotation types.

#### Python Annotation Types

In [ ]:
label = []
annotations_list = [
          checklist_annotation, 
          radio_annotation,
          bbox_annotation, 
          point_annotation, 
          polyline_annotation,
          global_checklist_annotation,
          global_radio_annotation
        #   video_mask_annotation
      ]

flatten_list_annotations = [ann for ann_sublist in annotations_list for ann in ann_sublist] 

label.append(
    lb_types.Label(
        data=lb_types.VideoData(global_key=global_key),
        annotations = flatten_list_annotations
    )
)


#### NDJSON annotations
Here we create the complete `label_ndjson` payload of annotations. There is one annotation for each *reference to an annotation* that we created above.

In [ ]:
## For nested frame base classifications we need to pass a featureSchemaId instead of the name. 

features = project.ontology().normalized

for i in features['tools']:
  print(i)
  if i['name'] == 'bbox_class':
    ## Classification feature schema id
    class_feature_schema_id = i['classifications'][0]['featureSchemaId']
    ## Answer feature schema id (select one of the answers)
    class_options_feature_schema_id = i['classifications'][0]['options'][0]['featureSchemaId']

    ## Update the original annotation with the schema ids
    for frame in frame_bbox_with_checklist_subclass_ndjson['segments']:
      for k in frame['keyframes']:
        k['classifications'][0].update(
            {'schemaId': class_feature_schema_id , 
              'answer': {'schemaId': class_options_feature_schema_id}
              }
            )
        

{'schemaNodeId': 'clfr7us7k05mb073508ye6jty', 'featureSchemaId': 'clfr7us7j05ma07356aeu0mr4', 'required': False, 'name': 'bbox_video', 'tool': 'rectangle', 'color': '#ff0000', 'archived': 0, 'classifications': []}
{'schemaNodeId': 'clfr7us7k05md0735c880dhb4', 'featureSchemaId': 'clfr7us7k05mc07351f9sbg7d', 'required': False, 'name': 'point_video', 'tool': 'point', 'color': '#cbff00', 'archived': 0, 'classifications': []}
{'schemaNodeId': 'clfr7us7k05mf07356tdc1udv', 'featureSchemaId': 'clfr7us7k05me073528is3w1a', 'required': False, 'name': 'line_video_frame', 'tool': 'line', 'color': '#00ff66', 'archived': 0, 'classifications': []}
{'schemaNodeId': 'clfr7us7k05mh0735abnaemoe', 'featureSchemaId': 'clfr7us7k05mg07351ret92w8', 'required': False, 'name': 'video_mask', 'tool': 'raster-segmentation', 'color': '#0066ff', 'archived': 0, 'classifications': []}
{'schemaNodeId': 'clfr7us7l05mr073515gk931a', 'featureSchemaId': 'clfr7us7k05mi073549i3ay0c', 'required': False, 'name': 'bbox_class', '

In [ ]:
label_ndjson = []


for annotation in [
    # point_annotation_ndjson,
    # bbox_annotation_ndjson,
    # polyline_frame_annotation_ndjson, 
    # frame_checklist_classification_ndjson, 
    # frame_radio_classification_ndjson,
    # nested_radio_classification,
    # nested_checklist_annotation_ndjson,
    # frame_bbox_with_checklist_subclass_ndjson,
    # global_radio_classification_ndjson,
    # global_checklist_classification_ndjson,
    video_mask_annotation_ndjson       
]:      
  annotation.update({
      'dataRow': {
          'globalKey': global_key
      }
  })
  label_ndjson.append(annotation)


### Step 6: Upload annotations to a project as pre-labels or completed labels
For the purpose of this tutorial only run one of the label imports at once, otherwise the previous import might get overwritten.

#### Model-Assisted Labeling (MAL)

In [ ]:
# Upload MAL label for this data row in project
upload_job_mal = lb.MALPredictionImport.create_from_objects(
    client = client, 
    project_id = project.uid, 
    name="mal_import_job-" + str(uuid.uuid4()), 
    predictions=label_ndjson)

upload_job_mal.wait_until_done();
print("Errors:", upload_job_mal.errors)
print("Status of uploads: ", upload_job_mal.statuses)
print("   ")

Errors: [{'uuid': '6769384c-fce2-4d3e-829b-cef2e13ae735', 'dataRow': {'id': 'clfco73at0080079n5dhm9y3a', 'globalKey': 'sample-video-2.mp4'}, 'status': 'FAILURE', 'errors': [{'name': 'InvalidAnnotation', 'message': 'Found empty mask', 'additionalInfo': None}]}]
Status of uploads:  [{'uuid': '6769384c-fce2-4d3e-829b-cef2e13ae735', 'dataRow': {'id': 'clfco73at0080079n5dhm9y3a', 'globalKey': 'sample-video-2.mp4'}, 'status': 'FAILURE', 'errors': [{'name': 'InvalidAnnotation', 'message': 'Found empty mask', 'additionalInfo': None}]}]
   


#### Label Import

In [ ]:
upload_job_label_import = lb.LabelImport.create_from_objects(
    client = client,
    project_id = project.uid, 
    name = "label_import_job-" + str(uuid.uuid4()),
    labels=label_ndjson
)

upload_job_label_import.wait_until_done()
print("Errors:", upload_job_label_import.errors)
print("Status of uploads: ", upload_job_label_import.statuses)
print("   ")

Errors: [{'uuid': 'a374c173-8aa7-4de8-894a-18b36f3e1fe8', 'dataRow': {'id': 'clfco73at0080079n5dhm9y3a', 'globalKey': 'sample-video-2.mp4'}, 'status': 'FAILURE', 'errors': [{'name': 'FetchImageError', 'message': 'Error while downloading image from url: https://api.labelbox.com/masks/feature/clfr7k2sx00033b6ja7dxd7ap/1', 'additionalInfo': None}]}]
Status of uploads:  [{'uuid': '68ca10d7-b0ba-4b3e-ae5c-a3877ec250c7', 'dataRow': {'id': 'clfco73at0080079n5dhm9y3a', 'globalKey': 'sample-video-2.mp4'}, 'status': 'SUCCESS'}, {'uuid': '161f866a-2734-49d6-8303-453e2c9da4fd', 'dataRow': {'id': 'clfco73at0080079n5dhm9y3a', 'globalKey': 'sample-video-2.mp4'}, 'status': 'SUCCESS'}, {'uuid': '70312492-7031-4a96-b19d-4ed25d968903', 'dataRow': {'id': 'clfco73at0080079n5dhm9y3a', 'globalKey': 'sample-video-2.mp4'}, 'status': 'SUCCESS'}, {'uuid': 'c948f83f-43f1-4a1e-9762-57d70981c6da', 'dataRow': {'id': 'clfco73at0080079n5dhm9y3a', 'globalKey': 'sample-video-2.mp4'}, 'status': 'SUCCESS'}, {'uuid': '08bc

### Optional deletions for cleanup

In [ ]:
# Delete Project
# project.delete()
# dataset.delete()
